## Classifying digits with convolutional neural networks

This notebook contains the solution to the MNIST activity. 

#### Load the data

Both Keras and TF-Learn contain the MNIST dataset that can be quickly loaded with some helper functions. This solution will use TF-Learn but the Keras solution will be commented out. The two libraries are very similar. 

In [26]:
# import numpy as np

import keras
from keras.datasets import mnist

# Load both the training and test data from Keras
# The X variables are the features and Y the ground truth categories 
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# we see that the training data has 60000 instances with 28x28 pixels
# and 10 categories, one for each digit
print("X_train.shape:", X_train.shape,"y_train.shape:", y_train.shape)

# X_train[0] is the first image and  X_train[0][0] is the first row of pixels
print("X_train[0][0]:", X_train[0][0])
# y_train[0] is the ground truth for the digit that X_train[0] represents
print("y_train[0]:", y_train[0])

X_train.shape: (60000, 28, 28) y_train.shape: (60000,)
X_train[0][0]: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
y_train[0]: 5


In [27]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# y_train is now a one-hot encoding of the ground truth
print("y_train.shape:", y_train.shape)
# first value an arry of ten binary digets with one corresponding to 5 the only 1
print("y_train[0]:", y_train[0])

y_train.shape: (60000, 10)
y_train[0]: [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


#### Build the ConvNet 

Create a small convolutional that will run on a CPU, so only use about 6 and 8 kernels in each convolutional layer. For the fully connected layer, just use a 32 to 64 units as well. We won't get state of the art performance but we don't want to wait all day for it to run. 

In [6]:
# -------------------------------------------------------------------------------------------
# Keras
import keras
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Input, Dense, Activation, Dropout, Flatten, merge
from keras.layers.convolutional import Conv2D, MaxPooling2D

#keras.callbacks.TensorBoard(log_dir='/tmp/tflearn_logs')


# 28x28 matrix of bits
cnn_input = Input(shape=(28, 28, 1), name='Input')

net = Conv2D(6, (3,3), activation='relu')(cnn_input)
net = MaxPooling2D(pool_size=(2,2))(net)

net = Conv2D(8, (3,3), activation='relu')(net)
net = MaxPooling2D(pool_size=(2,2))(net)

# keras uses a flatten layer when going from convolutional layers to normal
net = Flatten()(net)

net = Dense(32, activation='relu')(net)
net = Dropout(rate=0.5)(net)

out = Dense(10, activation='softmax')(net)
model = Model(inputs=cnn_input, outputs=out)
adam = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model.summary()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 6)         60        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 6)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 8)         440       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 8)           0         
_______________________________________________________

#### Train the network

Train the network and use the test data as the validation set. 

In [7]:

# Keras Train
model.fit(np.expand_dims(X_train, -1), y_train, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 17s 284us/step - loss: 2.7511 - acc: 0.3000
Epoch 2/5
60000/60000 [==============================] - 19s 309us/step - loss: 1.4389 - acc: 0.4734
Epoch 3/5
60000/60000 [==============================] - 19s 321us/step - loss: 1.0725 - acc: 0.6245
Epoch 4/5
60000/60000 [==============================] - 19s 322us/step - loss: 0.8350 - acc: 0.7153
Epoch 5/5
60000/60000 [==============================] - 19s 318us/step - loss: 0.7213 - acc: 0.7589


In [8]:
performance = model.evaluate(np.expand_dims(X_test, -1), y_test)

print()
print('Loss:', performance[0], 'Accuracy:', performance[1])

10000/10000 [==============================] - 1s 83us/step

Loss: 0.21377041118144988 Accuracy: 0.9611
